In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/software/multiPhATE2/PipelineOutput \
/media/concha-eloko/Linux/PPT_clean/in_vitro


In [1]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch
from torch import nn 
import torch.nn.functional as F

import os
import numpy as np
import tqdm
import matplotlib.pyplot as plt
from DepoScope_functions import Dpo_classifier , find_longest_non_zero_suite_with_n_zeros , predict_sequence, plot_token

import warnings
warnings.filterwarnings("ignore") 

path_work = "/media/concha-eloko/Linux/depolymerase_building"

esm2_model_path = f"{path_work}/esm2_t12_35M_UR50D-finetuned-depolymerase.labels_4/checkpoint-6015"
DpoDetection_path = f"{path_work}/DepoDetection.T12.4Labels.1908.model"

tokenizer = AutoTokenizer.from_pretrained(esm2_model_path)
esm2_finetuned = AutoModelForTokenClassification.from_pretrained(esm2_model_path)

/media/concha-eloko/Linux/conda_envs/ML_work/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_classifier = Dpo_classifier(esm2_finetuned) # Create an instance of Dpo_classifier
model_classifier.load_state_dict(torch.load(DpoDetection_path), strict = False) # Load the saved weights ; weird Error with some of the keys 
model_classifier.eval() # Set the model to evaluation mode for inference


Dpo_classifier(
  (pretrained_model): EsmForTokenClassification(
    (esm): EsmModel(
      (embeddings): EsmEmbeddings(
        (word_embeddings): Embedding(33, 480, padding_idx=1)
        (dropout): Dropout(p=0.0, inplace=False)
        (position_embeddings): Embedding(1026, 480, padding_idx=1)
      )
      (encoder): EsmEncoder(
        (layer): ModuleList(
          (0-11): 12 x EsmLayer(
            (attention): EsmAttention(
              (self): EsmSelfAttention(
                (query): Linear(in_features=480, out_features=480, bias=True)
                (key): Linear(in_features=480, out_features=480, bias=True)
                (value): Linear(in_features=480, out_features=480, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
                (rotary_embeddings): RotaryEmbedding()
              )
              (output): EsmSelfOutput(
                (dense): Linear(in_features=480, out_features=480, bias=True)
                (dropout): Dropout(p=0.0, inpla

In [4]:
path_out = "/media/concha-eloko/Linux/PPT_clean/in_vitro/Laura"
from Bio import SeqIO
from tqdm import tqdm 

prediction_results = {}
for phage in tqdm(os.listdir(path_out)) :
    fastas = SeqIO.parse(f"{path_out}/{phage}/{phage}_cgp_protein.faa" , "fasta")
    tmp_results = []
    for record in fastas :
        protein_seq = str(record.seq)
        if len(protein_seq) >= 200 :
            prediction, sequence_outputs = predict_sequence(model_classifier, protein_seq)
            if prediction[0] == 1 :
                a = (prediction , record.id)
                tmp_results.append(a)
            else :
                pass
    prediction_results[phage] = tmp_results

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [05:37<00:00,  8.23s/it]


In [5]:
prediction_results

{'Kpn13-P1_node2_spades': [],
 'Kpn26-P2_node1_spades': [],
 'Kpn13-P1_node1_spades': [],
 'Kpn26-P3_node1_spades': [((1.0, 1.0), 'cds3/-/1/2289')],
 'Kpn5-P1_node1_spades': [((1.0, 1.0), 'cds21/+/1/2376'),
  ((1.0, 1.0), 'cds26/+/1/3732')],
 'Kpn5-P2_node2_spades': [],
 'KPN10P1_node1_spades': [],
 'Kpn16-P2_node1_spades': [((1.0, 1.0), 'cds55/-/1/2526')],
 'Kpn10-P2_node1_spades': [((1.0, 1.0), 'cds32/+/1/2289')],
 'Kpn13-P2_node1_spades': [],
 'KPN25P1_node1_unicycler': [((1.0, 1.0), 'cds40/-/1/1983'),
  ((1.0, 1.0), 'cds52/-/1/2382')],
 'Kpn2-P4_node2_spades': [((1.0, 1.0), 'cds42/-/1/1080')],
 'Kpn1-P4_node2_spades': [((1.0, 1.0), 'cds42/-/1/1080')],
 'Kpn2-P1_node2_spades': [((1.0, 1.0), 'cds21/+/1/1080')],
 'Kpn30-P3_node1_unicycler': [],
 'Kpn24-P2_node1_spades': [],
 'Kpn7-P2_node1_spades': [((1.0, 1.0), 'cds40/-/1/4056')],
 'Kpn13-P2_node2_spades': [],
 'Kpn30-P2_node1_unicycler': [((1.0, 0.9999434947967529), 'cds11/-/1/3054'),
  ((1.0, 1.0), 'cds16/-/1/2430')],
 'Kpn16-P3_no

In [ ]:
dpos = [dpo[1] for file in prediction_results for dpo in prediction_results[file]]

for fasta_file in tqdm(os.listdir(path_out)) :
    fastas = SeqIO.parse(f"{path_out}/{fasta_file}" , "fasta")
    tmp_results = []
    for record in fastas :
        protein_seq = str(record.seq[0:-1].translate())
        if record.id in dpos and len(protein_seq) >= 200:
            print(record.id)
            prediction, sequence_outputs = predict_sequence(model_classifier, protein_seq)
            plot_token(sequence_outputs)
            print(protein_seq)
            print("\n")